In [1]:
import warnings
warnings.simplefilter('ignore')

import dill
import numpy as np
import pandas as pd
import requests
import shap

from lightfm import LightFM
from lightfm.data import Dataset
from lightgbm import LGBMRanker, LGBMClassifier
from rectools.metrics import calc_metrics, NDCG, MAP, Precision, Recall, MeanInvUserFreq
from rectools import Columns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from typing import Any, Dict, Tuple
from tqdm.auto import tqdm
from zipfile import ZipFile

from tools import generate_lightfm_recs_mapper, avg_user_metric

In [2]:
interactions = pd.read_csv("data_original/interactions.csv")
items = pd.read_csv("data_original/items.csv")
users = pd.read_csv("data_original/users.csv")

In [3]:
# Меняем названия колонок для использования rectools
interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'watched_pct': Columns.Weight,
    }, 
    inplace=True,
) 
# Меняем тип данных
interactions['datetime'] = interactions['datetime'].astype(np.datetime64)

# Заполняем пропуски
interactions_default_values: Dict[str, Any] = {
   Columns.Datetime: interactions[Columns.Datetime].median(),
    Columns.Weight: 0.,
    'total_dur': 0,
}
interactions.fillna(interactions_default_values, inplace=True)

# Смотрим что получилось
interactions.head(10)

,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5,1032142,6686,2021-05-13,11286,100.0
6,1016458,354,2021-08-14,1672,25.0
7,884009,693,2021-08-04,703,14.0
8,648682,1449,2021-06-13,26246,75.0
9,203219,13582,2021-08-22,6975,100.0


In [4]:
def encode_cat_cols(df: pd.DataFrame, cat_cols) -> Tuple[pd.DataFrame, Dict]:
    cat_col_encoding = {}  # словарь с категориями

    # Тут мы могли бы заполнять пропуски как еще одну категорию,
    # но они и так заполняются таким образом автоматически ниже
    # default_values = {col: 'None' for col in cat_cols}
    # df.fillna(default_values, inplace=True)

    for col in cat_cols:
        cat_col = df[col].astype('category').cat
        cat_col_encoding[col] = cat_col.categories
        df[col] = cat_col.codes.astype('category')
    return df, cat_col_encoding

users_cat_cols = [
    # 'user_id',
     'age', 'income', 'sex', 'kids_flg'
]
users, users_cat_col_encoding = encode_cat_cols(users, users_cat_cols)

# None уже кодируется как -1
users_cat_col_encoding['income'], users['income'].unique()

(Index(['income_0_20', 'income_150_inf', 'income_20_40', 'income_40_60',
        'income_60_90', 'income_90_150'],
       dtype='object'),
 [4, 2, 3, 0, -1, 5, 1]
 Categories (7, int64): [-1, 0, 1, 2, 3, 4, 5])

In [5]:
# Аналогичным образом кодируем категориальные колонки и пока удаляем текстовые
items_cat_cols = [
    # 'item_id', 
    'content_type', 'for_kids', 'studios',
]
items_text_cols = [
    'title', 'title_orig', 'genres', 'countries', 'directors', 'actors', 'description', 'keywords',
]
items_num_cols = [
    'release_year', 'age_rating', 
]
default_values_items = {
    'release_year': items['release_year'].median(),
    'age_rating': items['age_rating'].median(),
}

items, items_cat_col_encoding = encode_cat_cols(items, items_cat_cols) 
items = items.drop(items_text_cols, axis=1)
items.fillna(default_values_items, inplace=True)

items_cat_col_encoding['studios']

Index(['ABC', 'Amediateka', 'BBC', 'CBS', 'CBS All Access', 'Channel 4',
       'Cinemax', 'DAZN', 'Disney', 'Endemol', 'FX', 'Fox', 'Fremantle', 'HBO',
       'HBO Max', 'HBO, BBC', 'Legendary', 'MGM', 'New Regency Productions',
       'Paramount', 'Showtime', 'Sky', 'Sky, Fremantle', 'Sony Pictures',
       'Sony Pictures Television', 'Sony Pictures, рентв', 'Sony Plus',
       'Sony Plus, рентв', 'Starz', 'Universal', 'Universal, рентв',
       'Warner Bros', 'Warner Bros. Television', 'Ленфильм', 'Ленфильм, рентв',
       'Мосфильм', 'Рок фильм', 'рентв'],
      dtype='object')

In [6]:
max_date = interactions[Columns.Datetime].max()
min_date = interactions[Columns.Datetime].min()

print(f'min дата в interactions: {min_date}')
print(f'max дата в interactions: {max_date}')
print(f'Продолжительность: {max_date - min_date}')

min дата в interactions: 2021-03-13 00:00:00
max дата в interactions: 2021-08-22 00:00:00
Продолжительность: 162 days 00:00:00


In [7]:
# Обучать ранжирование будем на последнем месяце (30 дней) не считая отложенной недели
# Лучше зафиксировать временной диапазон если на проде планируется переобучать модель
ranker_days_count = 30

ranker_data = interactions[
    (interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=ranker_days_count))
]

# В дальнейшем ranker_data разбиваем по юзерам 
#  на train val test для обучения, валидации и тестирования ранкера
train_size = 0.7
val_size = 0.15
test_size = 0.15

# В train_test_split очень удобно можно сохранить исходное распределение по нужным факторам,
#  задав параметр stratify. Правда мы на это пока забьем)  

train_val_users, test_users = train_test_split(
    ranker_data['user_id'].unique(), random_state=42, test_size=test_size
)

train_users, val_users = train_test_split(
    train_val_users, random_state=42, test_size=val_size / (train_size + val_size)  # 15% от общего размера
)

In [8]:
# Для базовых моделей первого уровня (в нашем случае только lightfm) 
#  оставим все оставшиеся взаимодействия для обучения

base_models_data = interactions[
    (interactions[Columns.Datetime] < max_date - pd.Timedelta(days=ranker_days_count))
]

# Обучаем LightFM

In [9]:
# В рамках семинара мы не будем обучать большое количество моделей первого уровня и
# ограничимся только LightFM
lightfm_dataset = Dataset()
lightfm_user_ids = base_models_data['user_id'].unique()
lightfm_item_ids = base_models_data['item_id'].unique()
lightfm_dataset.fit(lightfm_user_ids, lightfm_item_ids)

In [10]:
# В качестве таргета можно придумать что-то сложное на основе имеющихся
# процента досмотра или абсолютного значения просмотра.
# Как один из вариантов - возьмем процент досмотра - watched_pct (Columns.Weight).

# Тогда матрицу интеракций и весов можно получить следующим образом:
interactions_matrix, weights_matrix = lightfm_dataset.build_interactions(
    zip(*base_models_data[['user_id', 'item_id', Columns.Weight]].values.T)
)
weights_matrix = weights_matrix.tocsr()

In [11]:
# Обучаем модель
# Не используем доступные фичи юзеров и айтемов, оставим это на этап реранжирования

lfm_model = LightFM(
    no_components=64, 
    learning_rate=0.01, 
    loss='warp', 
    # max_sampled=5, 
    random_state=42,
)

num_epochs = 3

for _ in tqdm(range(num_epochs)):
    lfm_model.fit_partial(weights_matrix)

  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
# Маппинги обычных айдишников во внутренние индексы lightfm для юзеров и айтемов
lightfm_mapping = lightfm_dataset.mapping()
lightfm_mapping = {
    'user_id_to_iid': lightfm_mapping[0],  # iid - internal lfm id
    'item_id_to_iid': lightfm_mapping[2],
}
# Маппинги внутренние индексов lightfm в обычные айдишники для юзеров и айтемов
lightfm_mapping['user_iid_to_id'] = {v: k for k, v in lightfm_mapping['user_id_to_iid'].items()}
lightfm_mapping['item_iid_to_id'] = {v: k for k, v in lightfm_mapping['item_id_to_iid'].items()}

In [13]:
# Топ 100 кандидатов 
# Как понять сколько их нужно?
top_N = 100

# Внутренние индексы юзеров и айтемов lightfm (индексы матрицы user-item)
user_lfm_index = np.array(list(lightfm_mapping['user_id_to_iid'].values()))
item_lfm_index = np.array(list(lightfm_mapping['item_id_to_iid'].values()))

# Делаем предикт, ограничиваем его сверху top_N 
# и сразу переводим в настоящие айдишники
mapper = generate_lightfm_recs_mapper(
    model=lfm_model, 
    N=top_N,
    item_iids=item_lfm_index, 
    user_id_to_iid=lightfm_mapping['user_id_to_iid'],
    item_iid_to_id=lightfm_mapping['item_iid_to_id'],
    known_item_ids=dict(),  # тут можно добавить уже просмотренный контент для его исключения
    num_threads=24,
)

In [14]:
# Генерируем предсказания и получаем скоры и ранги lightfm

candidates = pd.DataFrame({'user_id': lightfm_user_ids})
candidates['item_id'], candidates['lfm_score'] = zip(*candidates['user_id'].map(mapper))
candidates = candidates.explode(['item_id', 'lfm_score'], ignore_index=True)
candidates['lfm_rank'] = candidates.groupby('user_id').cumcount() + 1 

# Делаем чекпоинт - сохраняем кандидатов
candidates.to_csv('data_original/lfm_candidates.csv', index=False)

candidates.head(3)

,user_id,item_id,lfm_score,lfm_rank
0,176549,13865,4.600747,1
1,176549,9728,4.548446,2
2,176549,10440,4.349923,3


In [9]:
candidates = pd.read_csv("data_original/lfm_candidates.csv")

In [12]:
def calc_metrics_(candidates_df, rank_col: str) -> Dict[str, float]:
    metrics = {
        'ndcg@10': NDCG(k = 10),
        'map@10': MAP(k = 10),
        'Precision@10': Precision(k = 10),
        'recall@10': Recall(k = 10),
        'recall@100': Recall(k = 100),
        'novelty@10': MeanInvUserFreq(k = 10),
    }
    return calc_metrics(
        metrics=metrics,
        reco=(
            candidates_df
            .rename(columns={rank_col: Columns.Rank})
            [[Columns.User, Columns.Item, Columns.Rank]]
            [candidates_df[Columns.User].isin(test_users)]
        ),
        interactions=(
            ranker_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [ranker_data[Columns.User].isin(test_users)]
        ), 
        prev_interactions=(
            base_models_data
            [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
            [base_models_data[Columns.User].isin(test_users)]
        ),
        catalog=items['item_id'].unique()
    )

In [13]:
models_metrics: Dict[str, Dict[str, float]] = dict()
models_metrics['lfm'] = calc_metrics_(candidates, 'lfm_rank')
models_metrics['lfm']

{'Precision@10': 0.023946710051724402,
 'recall@10': 0.07968036767371942,
 'recall@100': 0.16477833207250517,
 'ndcg@10': 0.028053126566969854,
 'map@10': 0.03547440909889698,
 'novelty@10': 2.9414476202455564}

# Обучаем Популярные

In [30]:
# POPULAR
from rectools.dataset import Interactions, Dataset

interactions_class = Interactions(interactions)

interactions_class.df.head()

,user_id,item_id,datetime,total_dur,weight
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [32]:
from rectools.models.popular import PopularModel

pop_model = PopularModel()
dataset = Dataset.construct(
    interactions_df=interactions_class.df,
    user_features_df=None,
    item_features_df=None,
)
catalog = interactions_class.df[Columns.Item].unique()
pop_model.fit(dataset)

In [41]:
recos = pop_model.recommend(
    users=list(set(interactions_class.df.user_id.unique()) - set(lightfm_user_ids)),
    dataset=dataset,
    k=100,
    filter_viewed=True,
)

In [43]:
recos.to_csv("data_original/pop_candidates.csv", index=False)

In [14]:
recos = pd.read_csv("data_original/pop_candidates.csv")

# Готовим фичи

In [16]:
# Получаем длину истории юзера 
base_models_data['user_hist'] = (
    base_models_data.groupby('user_id')
    ['item_id'].transform('count')
)
# Получаем популярность контента
base_models_data['item_pop'] = (
    base_models_data.groupby('item_id')
    ['user_id'].transform('count')
)
# Получаем среднюю популярность контента, просматриваемого этим юзером
base_models_data['user_avg_pop'] = (
    base_models_data.groupby('user_id')
    ['item_pop'].transform('mean')
)
# Получаем среднюю длину истории пользователя, которые смотрит этот контент
base_models_data['item_avg_hist'] = (
    base_models_data.groupby('item_id')
    ['user_hist'].transform('mean')
)
# Получаем популярность последнего просмотренного контента
base_models_data.sort_values(
    by=[Columns.User, Columns.Datetime], 
    ascending=[True, False], 
    ignore_index=True,
    inplace=True,
)
base_models_data['user_last_pop'] = (
    base_models_data.groupby('user_id')
    ['item_pop'].transform('first')
)
base_models_data.head(3)

,user_id,item_id,datetime,total_dur,weight,user_hist,item_pop,user_avg_pop,item_avg_hist,user_last_pop
0,0,6006,2021-07-20,1,0.0,6,5208,41885.0,16.891897,5208
1,0,7102,2021-07-19,169,3.0,6,11626,41885.0,20.349475,5208
2,0,14359,2021-07-19,130,2.0,6,6053,41885.0,22.546836,5208


In [17]:
# Добавляем новые фичи в соответствующие таблички
items = pd.merge(
    left=items, 
    right=(
        base_models_data
        [['item_id', 'item_pop', 'item_avg_hist']]
        .drop_duplicates()
    ),
    how='left',
    on='item_id',
)

users = pd.merge(
    left=users, 
    right=(
        base_models_data
        [['user_id', 'user_hist', 'user_avg_pop', 'user_last_pop']]
        .drop_duplicates()
    ),
    how='left',
    on='user_id',
)
users.head(3)

,user_id,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop
0,973171,1,4,1,1,5.0,19550.800000,93403.0
1,962099,0,2,1,0,13.0,1329.307692,260.0
2,1047345,3,3,0,0,NaN,NaN,NaN


In [18]:
# Обновляем дефолтные значения
# Прямо сейчас обновлять таблички users и items не обязательно, 
# сделаем это при джойне с кандидатами

# Для новых фичей айтемов
default_values_items['item_pop'] = base_models_data['item_pop'].median()
default_values_items['item_avg_hist'] = base_models_data['item_avg_hist'].median()

# Для новых фичей юзеров
default_values_users = {
    'user_hist': 0,
    'user_avg_pop': base_models_data['user_avg_pop'].median(),
    'user_last_pop': base_models_data['user_last_pop'].median(),
}

# объединяем кандидатов из ЛФМ и Популярное для дальнейшего обучения ранкера

Убрал колонку скор, так как скоры из лфм и популярного имеют разное происхождение

In [19]:
candidates = pd.concat([candidates.drop("lfm_score", axis=1).rename(columns={"lfm_rank": "first_rank"}), recos.drop("score", axis=1).rename(columns={"rank": "first_rank"})])
candidates.to_csv("data_original/pop_lfm_cand.csv", index=False)
candidates.head()

,user_id,item_id,first_rank
0,176549,13865,1
1,176549,9728,2
2,176549,10440,3
3,176549,3734,4
4,176549,4151,5


In [17]:
candidates = pd.read_csv("data_original/pop_lfm_cand.csv").reset_index(drop=True)
# POPULAR

# Собираем датасет для ранкера

In [20]:
# Вспоминаем про наши выборки интеракций для ранкера.
# Мы отобрали юзеров для обучения, валидации и теста.
# Оставляем среди них только тех, для кого есть и рекомы и таргеты

def users_filter(
    user_list: np.ndarray,
    candidates_df: pd.DataFrame, 
    df: pd.DataFrame,
) -> pd.DataFrame:
    # Джойним интеракции на наших кандидатов для юзеров из трейна, вал и теста
    df = pd.merge(
        df[df['user_id'].isin(user_list)], 
        candidates_df[candidates_df['user_id'].isin(user_list)], 
        how='right',  # right ? 
        on=['user_id', 'item_id']
    )
    # Проставляем дефолтные значения интеракций
    # min_score: float =  df['lfm_score'].min() - 0.01
    max_rank: int = df['first_rank'].max() + 1  # 101
    
    default_values = {
        'first_rank': max_rank,
        # Важно использовате те же дефолтные значения для интеракций, 
        # чтобы не сделать утечку
        **interactions_default_values,
    }
    df.fillna(default_values, inplace=True)
        
    # Сортируем по user_id - это пригодится для вычисления рангов и групп для ранжирования
    df.sort_values(
        by=['user_id', 'item_id'],
        inplace=True,
    )
    return df

ranker_train = users_filter(train_users, candidates, ranker_data)
ranker_val = users_filter(val_users, candidates, ranker_data)
ranker_test = users_filter(test_users, candidates, ranker_data)

ranker_train.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank
11733086,3,14,2021-07-01,0.0,0.0,87
11733087,3,24,2021-07-01,0.0,0.0,88
11733036,3,101,2021-07-01,0.0,0.0,37


In [21]:
# Добавляем фичи
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(
        df, 
        users, 
        how='left', 
        on=['user_id']
    )
    df = pd.merge(
        df, 
        items, 
        how='left', 
        on=['item_id']
    )

    # При джойне могут получиться строки с несуществующими айтемами или юзерами.
    # Надо заполнить пропуски. Используем заготовленные дефолтные значения,
    # чтобы не сделать утечку
    df.fillna(default_values_items, inplace=True)
    df.fillna(default_values_users, inplace=True)

    # Категориальные фичи закодированы пандасом так, что None === -1
    # Если изначально пропусков не было, то нужно добавить такое значение категории
    for col in df.columns:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            if -1 not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories(-1)
            df.fillna({col: -1}, inplace=True)
    return df

# Не забываем добавить фичи в трейн, вал и тест
# Еще правильнее бы было сначала подготовить датасет, 
# а потом его разбивать по юзерам - так бы мы избежали дублирования операций.
ranker_train = add_features(ranker_train)
ranker_val = add_features(ranker_val)
ranker_test = add_features(ranker_test)

ranker_train.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,user_hist,user_avg_pop,user_last_pop,content_type,release_year,for_kids,age_rating,studios,item_pop,item_avg_hist
0,3,14,2021-07-01,0.0,0.0,87,-1,-1,-1,-1,0.0,11957.864865,2858.0,1,2019.0,-1,16.0,-1,5675.0,17.167225
1,3,24,2021-07-01,0.0,0.0,88,-1,-1,-1,-1,0.0,11957.864865,2858.0,1,2020.0,-1,16.0,-1,6676.0,17.415518
2,3,101,2021-07-01,0.0,0.0,37,-1,-1,-1,-1,0.0,11957.864865,2858.0,0,2019.0,-1,18.0,-1,9542.0,17.990673


In [22]:
# Датасеты готовы, остались только таргеты, 
# которые можно посчитать на основе колонок total_dur и watched_pct

# Делаем еще один чекпоинт.

# Не пользуемся методом eval если точно не знаем, что за строка.
# Он не безопасен и долго работает.
# Можно заменить на locals()[name]
for name in ['train', 'val', 'test']:
    path: str = f'data_original/lfm_pop_ranker_{name}.csv'
    # eval(f'ranker_{name}').to_csv(path, index=False)
    locals()[f'ranker_{name}'].to_csv(path, index=False)

In [2]:
# Загружаем данные
for name in ['train', 'val', 'test']:
    path: str = f'data_original/lfm_pop_ranker_{name}.csv'
    locals()[f'ranker_{name}'] = pd.read_csv(path)

In [23]:
def get_group(df: pd.DataFrame) -> np.ndarray:
    return np.array(
        df[['user_id', 'item_id']]
        .groupby(by=['user_id']).count()
        ['item_id']
    )

# Добавим таргет посложнее

def add_target(df: pd.DataFrame) -> pd.DataFrame:
    """
    0 - доля досмотра < 0.15
    1 - 0.15 <= доля досмотра < 0.75
    2 - 0.75 <= доля досмотра
    """
    df['target_ranker'] = (df[Columns.Weight] >= 15).astype(int)  # 'watched_pct'
    df['target_ranker'] += (df[Columns.Weight] >= 75).astype(int)
    return df

ranker_train = add_target(ranker_train)
ranker_val = add_target(ranker_val)
ranker_test = add_target(ranker_test)

ranker_train.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,user_avg_pop,user_last_pop,content_type,release_year,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker
0,3,14,2021-07-01,0.0,0.0,87,-1,-1,-1,-1,...,11957.864865,2858.0,1,2019.0,-1,16.0,-1,5675.0,17.167225,0
1,3,24,2021-07-01,0.0,0.0,88,-1,-1,-1,-1,...,11957.864865,2858.0,1,2020.0,-1,16.0,-1,6676.0,17.415518,0
2,3,101,2021-07-01,0.0,0.0,37,-1,-1,-1,-1,...,11957.864865,2858.0,0,2019.0,-1,18.0,-1,9542.0,17.990673,0


In [24]:
# Убираем ненужные айдишники, временные метки и таргеты.
# Для обучения используются только cols:
cols = [
    'first_rank', 
    'age', 'income', 'sex', 'kids_flg', 'user_hist', 'user_avg_pop', 'user_last_pop',
    'content_type', 'release_year', 'for_kids', 'age_rating', 'studios', 'item_pop', 'item_avg_hist',
]
# Из них категориальные:
cat_cols = [
    'age', 'income', 'sex', 'kids_flg',
    'content_type', 'for_kids', 'studios',
]

early_stopping_rounds = 32
params = {
    'objective': 'lambdarank',  # lambdarank, оптимизирующий ndcg 
    'n_estimators': 10000,  # максимальное число деревьев
    'max_depth': 4,  # максимальная глубина дерева
    'num_leaves': 10,  # число листьев << 2^max_depth
    'min_child_samples': 100,  # число примеров в листе
    'learning_rate': 0.25,  # шаг обучения
    'reg_lambda': 1,  # L2 регуляризация
    'colsample_bytree': 0.9,  # доля колонок, которая используется в каждом дереве
    'early_stopping_rounds': early_stopping_rounds,  # число итераций, в течение которых нет улучшения метрик
    'verbose': early_stopping_rounds // 8,  # период вывода метрик
    'random_state': 42,
}
fit_params = {
    'X': ranker_train[cols],
    'y': ranker_train['target_ranker'],
    'group': get_group(ranker_train),
    'eval_set': [(ranker_val[cols], ranker_val['target_ranker'])],
    'eval_group': [get_group(ranker_val)],
    'eval_metric': 'ndcg',
    'eval_at': (3, 5, 10),
    'categorical_feature': cat_cols,
    'feature_name': cols,
}
listwise_model = LGBMRanker(**params)
listwise_model.fit(**fit_params)

[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.808345
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.262309
[LightGBM] [Debug] init for col-wise cost 0.196873 seconds, init for row-wise cost 0.891867 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.297636 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 1166
[LightGBM] [Info] Number of data points in the train set: 30404700, number of used features: 15
[LightGBM] [Warning] early_stopping_round is set=32, early_stopping_rounds=32 will be ignored. Current value: early_stopping_round=32
[LightGBM] [Debug] Trained a tree with leaves = 10 and depth = 4
Training

LGBMRanker(colsample_bytree=0.9, early_stopping_rounds=32, learning_rate=0.25,
           max_depth=4, min_child_samples=100, n_estimators=10000,
           num_leaves=10, objective='lambdarank', random_state=42, reg_lambda=1,
           verbose=4)

In [25]:
def add_score_and_rank(df: pd.DataFrame, y_pred_scores: np.ndarray, name: str) -> pd.DataFrame:
    # Добавляем скор модели второго уровня
    df[f'{name}_score'] = y_pred_scores
    # Добавляем ранг модели второго уровня
    df.sort_values(
        by=['user_id', f'{name}_score'],
        ascending=[True, False],
        inplace=True,
    )
    df[f'{name}_rank'] = df.groupby('user_id').cumcount() + 1

    # Исключаем айтемы, которые не были предсказаны на первом уровне
    mask = (df['first_rank'] < 101).to_numpy()
    # Добавляем общий скор двухэтапной модели
    eps: float = 0.001
    min_score: float = min(y_pred_scores) - eps
    df[f'{name}_hybrid_score'] = df[f'{name}_score'] * mask
    df[f'{name}_hybrid_score'].replace(
        0,
        min_score,
        inplace=True,
    )
    # Добавляем общий ранг двухэтапной модели
    df[f'{name}_hybrid_rank'] = df[f'{name}_rank'] * mask
    max_rank: int = 101
    df[f'{name}_hybrid_rank'].replace(
        0,
        max_rank,
        inplace=True,
    )
    return df

In [26]:
y_pred: np.ndarray = listwise_model.predict(ranker_test[cols])
ranker_test = add_score_and_rank(ranker_test, y_pred, 'listwise')
ranker_test.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker,listwise_score,listwise_rank,listwise_hybrid_score,listwise_hybrid_rank
92,1,15297,2021-07-01,0.0,0.0,1,1,2,0,1,...,-1,18.0,-1,137128.0,7.364295,0,2.240066,1,2.240066,1
57,1,9728,2021-07-01,0.0,0.0,2,1,2,0,1,...,-1,18.0,-1,76978.0,11.165736,0,1.606806,2,1.606806,2
96,1,16228,2021-07-01,0.0,0.0,29,1,2,0,1,...,0,18.0,-1,7322.0,9.994127,0,1.184456,3,1.184456,3


In [29]:
models_metrics['listwise'] = calc_metrics_(ranker_test, 'listwise_rank')
models_metrics['listwise_hybrid'] = calc_metrics_(ranker_test, 'listwise_hybrid_rank')
pd.DataFrame(models_metrics)[['lfm', 'listwise', 'listwise_hybrid']]

,lfm,listwise,listwise_hybrid
Precision@10,0.023947,0.028608,0.028608
recall@10,0.079680,0.095398,0.095398
recall@100,0.164778,0.164778,0.164778
ndcg@10,0.028053,0.034574,0.034574
map@10,0.035474,0.045559,0.045559
novelty@10,2.941448,3.926457,3.926457


# Применяем модели на всех пользователях

In [31]:
y_pred_train: np.ndarray = listwise_model.predict(ranker_train[cols])
ranker_train = add_score_and_rank(ranker_train, y_pred_train, 'listwise')
ranker_train.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker,listwise_score,listwise_rank,listwise_hybrid_score,listwise_hybrid_rank
90,3,15297,2021-07-01,0.0,0.0,1,-1,-1,-1,-1,...,-1,18.0,-1,137128.0,7.364295,0,2.472905,1,2.472905,1
60,3,10440,2021-07-23,44827.0,90.0,2,-1,-1,-1,-1,...,-1,18.0,-1,141889.0,8.068716,2,1.485018,2,1.485018,2
56,3,9728,2021-07-23,10448.0,100.0,4,-1,-1,-1,-1,...,-1,18.0,-1,76978.0,11.165736,2,1.406217,3,1.406217,3


In [32]:
y_pred_val: np.ndarray = listwise_model.predict(ranker_val[cols])
ranker_val = add_score_and_rank(ranker_val, y_pred_val, 'listwise')
ranker_val.head(3)

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker,listwise_score,listwise_rank,listwise_hybrid_score,listwise_hybrid_rank
60,21,9728,2021-07-01,0.0,0.0,3,3,2,0,0,...,-1,18.0,-1,76978.0,11.165736,0,1.191978,1,1.191978,1
94,21,15297,2021-07-01,0.0,0.0,4,3,2,0,0,...,-1,18.0,-1,137128.0,7.364295,0,1.138138,2,1.138138,2
65,21,10440,2021-07-01,0.0,0.0,2,3,2,0,0,...,-1,18.0,-1,141889.0,8.068716,0,0.956449,3,0.956449,3


In [35]:
all_preds = pd.concat([ranker_train, ranker_val, ranker_test])
all_preds.head()

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker,listwise_score,listwise_rank,listwise_hybrid_score,listwise_hybrid_rank
90,3,15297,2021-07-01,0.0,0.0,1,-1,-1,-1,-1,...,-1,18.0,-1,137128.0,7.364295,0,2.472905,1,2.472905,1
60,3,10440,2021-07-23,44827.0,90.0,2,-1,-1,-1,-1,...,-1,18.0,-1,141889.0,8.068716,2,1.485018,2,1.485018,2
56,3,9728,2021-07-23,10448.0,100.0,4,-1,-1,-1,-1,...,-1,18.0,-1,76978.0,11.165736,2,1.406217,3,1.406217,3
22,3,3734,2021-07-01,0.0,0.0,7,-1,-1,-1,-1,...,-1,16.0,-1,50004.0,12.134949,0,1.135074,4,1.135074,4
80,3,13865,2021-07-01,0.0,0.0,3,-1,-1,-1,-1,...,-1,12.0,-1,93403.0,10.408520,0,1.043227,5,1.043227,5


In [37]:
all_preds[all_preds.user_id == 3][["user_id", "item_id", "first_rank", "listwise_hybrid_score", "listwise_hybrid_rank"]]

,user_id,item_id,first_rank,listwise_hybrid_score,listwise_hybrid_rank
90,3,15297,1,2.472905,1
60,3,10440,2,1.485018,2
56,3,9728,4,1.406217,3
22,3,3734,7,1.135074,4
80,3,13865,3,1.043227,5
...,...,...,...,...,...
6,3,849,25,-3.007452,96
50,3,8373,70,-3.022558,97
58,3,10077,71,-3.033870,98
21,3,3402,91,-3.247690,99


In [38]:
metrics = {
    'ndcg@10': NDCG(k = 10),
    'map@10': MAP(k = 10),
    'Precision@10': Precision(k = 10),
    'recall@10': Recall(k = 10),
    'recall@100': Recall(k = 100),
    'novelty@10': MeanInvUserFreq(k = 10),
}
calc_metrics(
    metrics=metrics,
    reco=(
        all_preds
        .rename(columns={"listwise_hybrid_rank": Columns.Rank})
        [[Columns.User, Columns.Item, Columns.Rank]]
    ),
    interactions=(
        ranker_data
        [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
    ), 
    prev_interactions=(
        base_models_data
        [[Columns.User, Columns.Item, Columns.Datetime, Columns.Weight]]
    ),
    catalog=items['item_id'].unique()
)

{'Precision@10': 0.02852426482607465,
 'recall@10': 0.09631375999923786,
 'recall@100': 0.16557661456021758,
 'ndcg@10': 0.03464958706338471,
 'map@10': 0.04642227154785284,
 'novelty@10': 4.718308273026473}

In [39]:
all_preds

,user_id,item_id,datetime,total_dur,weight,first_rank,age,income,sex,kids_flg,...,for_kids,age_rating,studios,item_pop,item_avg_hist,target_ranker,listwise_score,listwise_rank,listwise_hybrid_score,listwise_hybrid_rank
90,3,15297,2021-07-01,0.0,0.0,1,-1,-1,-1,-1,...,-1,18.0,-1,137128.0,7.364295,0,2.472905,1,2.472905,1
60,3,10440,2021-07-23,44827.0,90.0,2,-1,-1,-1,-1,...,-1,18.0,-1,141889.0,8.068716,2,1.485018,2,1.485018,2
56,3,9728,2021-07-23,10448.0,100.0,4,-1,-1,-1,-1,...,-1,18.0,-1,76978.0,11.165736,2,1.406217,3,1.406217,3
22,3,3734,2021-07-01,0.0,0.0,7,-1,-1,-1,-1,...,-1,16.0,-1,50004.0,12.134949,0,1.135074,4,1.135074,4
80,3,13865,2021-07-01,0.0,0.0,3,-1,-1,-1,-1,...,-1,12.0,-1,93403.0,10.408520,0,1.043227,5,1.043227,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6515275,1097529,12463,2021-07-01,0.0,0.0,76,3,4,0,0,...,-1,18.0,-1,7447.0,19.088492,0,-2.745267,96,-2.745267,96
6515207,1097529,849,2021-07-01,0.0,0.0,37,3,4,0,0,...,-1,18.0,-1,13304.0,16.444678,0,-2.847674,97,-2.847674,97
6515298,1097529,16361,2021-07-01,0.0,0.0,80,3,4,0,0,...,-1,18.0,-1,7401.0,19.266586,0,-2.966489,98,-2.966489,98
6515252,1097529,8447,2021-07-01,0.0,0.0,99,3,4,0,0,...,-1,18.0,-1,5828.0,23.066404,0,-3.010986,99,-3.010986,99


In [41]:
final_recos = all_preds.groupby("user_id")["item_id"].agg(lambda x: list(x)[:10])
final_recos.head()

user_id
1     [15297, 9728, 16228, 3734, 13865, 7829, 12192,...
3     [15297, 10440, 9728, 3734, 13865, 16228, 4151,...
9     [16228, 7829, 12192, 6809, 7571, 8636, 7793, 4...
11    [15297, 10440, 9728, 13865, 3734, 12192, 4151,...
12    [15297, 9728, 10440, 3734, 16228, 12192, 13865...
Name: item_id, dtype: object

In [45]:
import json

with open("double.json", "w") as file:
    json.dump(final_recos.to_dict(), file)